<a href="https://colab.research.google.com/github/FelipeAMiehrig/MLOps_day2/blob/master/modelling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import datetime
import arviz as az
az.style.use("arviz-darkgrid")
import random

random.seed(10)

In [ ]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
link = 'https://drive.google.com/file/d/1rwc63vMSOJ9IT5Cuzzb9HKK_PCpoRHoI/view?usp=sharing'
fluff, id = link.split('=')
print (id) # Verify that you have everything after '='
downloaded = drive.CreateFile({'id':'1rwc63vMSOJ9IT5Cuzzb9HKK_PCpoRHoI'}) 
downloaded.GetContentFile('df_with_cluster_weatherstack.csv')  
df = pd.read_csv('df_with_cluster_weatherstack.csv', index_col = 0,  parse_dates= ['created_at',	'finished_at'])
# Dataset is now stored in a Pandas Dataframe

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

In [ ]:
!pip install numpyro

In [ ]:
df = pd.read_csv('df_with_cluster_weatherstack.csv', index_col = 0,  parse_dates= ['created_at',	'finished_at'])
df.pop(df.columns[0])

In [ ]:
df.columns

## some feature engineering

In [ ]:
df['weekday'] = df['created_at'].apply(lambda x: x.weekday()).astype('category')
df['seconds_day'] = df['created_at'].apply(lambda x: x.hour*60*60 + x.minute*60+x.second)
seconds_in_day = 24*60*60
df['hourday'] = df['created_at'].apply(lambda x: x.hour)
df['timeday'] = df['hourday'].apply(lambda x: 0 if x < 12 else (1 if x < 18 else 3)).astype('category')
df['rain'] = df['prcp'].apply(lambda x: 1 if x > 0 else 0)
df['rain2'] = df['precip'].apply(lambda x: 1 if x > 0 else 0)
df['snow'] = df.snow.apply(lambda x: x if x==x else 0)
df['daymonth'] = df['created_at'].apply(lambda x: x.day)
df['weekday_cont'] = df['created_at'].apply(lambda x: x.weekday()+1)
df['weekday_hour'] = df['created_at'].apply(lambda x: (x.weekday())*24 + x.hour)
df['weekday_hour_sin'] = np.sin(2*np.pi*df['weekday_hour']/(24*7))
df['weekday_hour_cos'] = np.cos(2*np.pi*df['weekday_hour']/(24*7))

In [ ]:
import math

def map_coordinates(abbreviation, radius):
    coordinates = {
        'N': (0, radius),
        'NNE': (math.sin(math.pi / 8) * radius, math.cos(math.pi / 8) * radius),
        'NE': (math.sin(math.pi / 4) * radius, math.cos(math.pi / 4) * radius),
        'ENE': (math.sin(3 * math.pi / 8) * radius, math.cos(3 * math.pi / 8) * radius),
        'E': (radius, 0),
        'ESE': (math.sin(5 * math.pi / 8) * radius, -math.cos(5 * math.pi / 8) * radius),
        'SE': (math.sin(3 * math.pi / 4) * radius, -math.cos(3 * math.pi / 4) * radius),
        'SSE': (math.sin(7 * math.pi / 8) * radius, -math.cos(7 * math.pi / 8) * radius),
        'S': (0, -radius),
        'SSW': (-math.sin(7 * math.pi / 8) * radius, -math.cos(7 * math.pi / 8) * radius),
        'SW': (-math.sin(3 * math.pi / 4) * radius, -math.cos(3 * math.pi / 4) * radius),
        'WSW': (-math.sin(5 * math.pi / 8) * radius, -math.cos(5 * math.pi / 8) * radius),
        'W': (-radius, 0),
        'WNW': (-math.sin(3 * math.pi / 8) * radius, math.cos(3 * math.pi / 8) * radius),
        'NW': (-math.sin(math.pi / 4) * radius, math.cos(math.pi / 4) * radius),
        'NNW': (-math.sin(math.pi / 8) * radius, math.cos(math.pi / 8) * radius)
    }

    return coordinates.get(abbreviation, (0, 0))

In [ ]:
radius_sin = df.wind_dir.apply(lambda x: map_coordinates(x,1)[0])
radius_cos = df.wind_dir.apply(lambda x: map_coordinates(x,1)[1])
df['radius_sin_wind_dir'] = radius_sin
df['radius_cos_wind_dir'] = radius_cos

In [ ]:
def sample(df, column, random_state = 42): 
  classes = list(set(df[column].values))
  if len(df[df[column]==classes[0]]) < len(df[df[column]==classes[1]]):
    minority_class = classes[0]
    majority_class = classes[1]
  else: 
    minority_class = classes[1]
    majority_class = classes[0]
  minority_df = df[df[column]==minority_class]
  n_min = len(minority_df)
  majority_df  = df[df[column]==majority_class].sample(n= n_min, random_state=random_state)
  balanced_df = pd.concat([minority_df, majority_df]).sample(frac=1, random_state=random_state).reset_index(drop=True)
  return balanced_df


In [ ]:
sampled_df = sample(df, 'vehicle_type')

## checking dataset

In [ ]:
plt.hist(sampled_df.vehicle_type)
plt.title('final_df')
plt.show()

In [ ]:
grouped = sampled_df.groupby('cluster').count()[['member_type']].sort_values(by='member_type',ascending=False)

In [ ]:
grouped.hist(bins=30)

In [ ]:
fig = px.scatter_mapbox(df, lat="user_location_latitude", lon="user_location_longitude", hover_name="minutes", hover_data=[ 'cluster_departure'],
                        color_discrete_sequence=["fuchsia"], zoom=3, height=300)
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()
fig.show()

## trying logistic regression via frequentist approach

In [ ]:
df.columns

In [ ]:

# importing libraries
import statsmodels.api as sm
from statsmodels.formula.api import logit

sampled_df['vehicle_binary'] = (sampled_df["vehicle_type"] != 'bike').astype(float)
# building the model and fitting the data
logistic = logit( "vehicle_binary ~ member_type + precip+feelslike+ wind_speed  + kilometers +cloudcover+ uv_index+  radius_sin_wind_dir + radius_cos_wind_dir+humidity+weekday_hour_sin + weekday_hour_cos", sampled_df).fit()

In [ ]:

print(logistic.summary())

In [ ]:
x = np.arange(0,24*7+1,1)

sinus = np.array([np.sin(2*np.pi*i/(24*7)) for i in x])
cosinus =  np.array([np.cos(2*np.pi*i/(24*7)) for i in x])
sincos = 0.0987*sinus -0.2114*cosinus
plt.plot(x, sincos)
plt.show()

In [ ]:
!pip install jax

In [ ]:
import jax.numpy as jnp
from jax import nn, random, vmap
from jax.scipy.special import expit

import numpyro
import numpyro.distributions as dist
import numpyro.optim as optim
from numpyro.diagnostics import print_summary
from numpyro.infer import MCMC, NUTS, Predictive, SVI, Trace_ELBO, log_likelihood
from numpyro.infer.autoguide import AutoLaplaceApproximation

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

In [ ]:
sampled_df = sampled_df.dropna()

## getting rid of clusters with zero observations due to resampling and relabeling them

In [ ]:
sampled_df.sort_values('cluster')

In [ ]:
old_cluster = sampled_df.groupby('cluster').count().index.values
cluster_to_join = pd.DataFrame({'old_cluster':old_cluster, 'new_cluster':np.arange(0,len(old_cluster))})
sampled_df = sampled_df.merge(cluster_to_join, how='inner', left_on='cluster', right_on='old_cluster')

In [ ]:
cluster_to_join

In [ ]:
import multiprocessing
multiprocessing.cpu_count()
num_chains = multiprocessing.cpu_count() 

## Logistic regression: Member type

In [ ]:
dat_list = dict(
    vehicle = (sampled_df["vehicle_type"] != 'bike').astype(float).values,
    member_type = OrdinalEncoder().fit_transform(sampled_df.member_type.values.reshape(-1,1)).reshape(-1).astype(int)
)

def model(member_type, vehicle=None):
    a = numpyro.sample("a", dist.Normal(0, 1.5).expand([3]))
    logit_p = a[member_type]
    numpyro.sample("vehicle", dist.Bernoulli(logits=logit_p), obs=vehicle)

m1 = MCMC(NUTS(model), num_warmup =1000, num_samples=2000, num_chains = 4)
m1.run(random.PRNGKey(0), vehicle=dat_list['vehicle'], member_type = dat_list['member_type'])
m1.print_summary(0.95)

In [ ]:
az.plot_pair(az.from_numpyro(m1))
plt.show()

In [ ]:
az.plot_trace(az.from_numpyro(m1))
plt.show()

In [ ]:
az.plot_rank(az.from_numpyro(m1))
plt.show()

## Logistic Regression: member type and weekday

In [ ]:

dat_list = dict(
    vehicle = (sampled_df["vehicle_type"] != 'bike').astype(float).values,
    member_type = OrdinalEncoder().fit_transform(sampled_df.member_type.values.reshape(-1,1)).reshape(-1).astype(int),
    weekday_hour_sin = sampled_df.weekday_hour_sin.values,
    weekday_hour_cos = sampled_df.weekday_hour_cos.values

)



def model2(member_type, weekday_hour_sin,weekday_hour_cos , vehicle=None):
    a = numpyro.sample("a", dist.Normal(0, 0.5).expand([3]))
    bweekday_sin = numpyro.sample("bweekdaysin", dist.Normal(0, 0.5))
    bweekday_cos = numpyro.sample("bweekdaycos", dist.Normal(0, 0.5))
    logit_p =  bweekday_sin*weekday_hour_sin + bweekday_cos*weekday_hour_cos  + a[member_type]
    numpyro.sample("vehicle", dist.Bernoulli(logits=logit_p), obs=vehicle)

m2 = MCMC(NUTS(model2), num_warmup =1000, num_samples=2000, num_chains = 4)
m2.run(random.PRNGKey(0), **dat_list)
m2.print_summary(0.95)

In [ ]:
az.plot_rank(az.from_numpyro(m2))
plt.show()

In [ ]:
az.plot_trace(az.from_numpyro(m2))
plt.show()

In [ ]:
m11_2 = AutoLaplaceApproximation(model2)
svi = SVI(
    model2,
    m11_2,
    optim.Adam(1),
    Trace_ELBO(),
    **dat_list
)
svi_result = svi.run(random.PRNGKey(0), 1000)
p11_2 = svi_result.params

In [ ]:
p11_2

##Logistic Regression: member type, temperature and wind speed

In [ ]:
dat_list = dict(
    vehicle = (sampled_df["vehicle_type"] != 'bike').astype(float).values,
    member_type = OrdinalEncoder().fit_transform(sampled_df.member_type.values.reshape(-1,1)).reshape(-1).astype(int),
    feelslike = sampled_df.feelslike.values,
    wind_speed = sampled_df.wind_speed.values,
    weekday_hour_sin = sampled_df.weekday_hour_sin.values,
    weekday_hour_cos = sampled_df.weekday_hour_cos.values
)

def model3(member_type, weekday_hour_sin,weekday_hour_cos ,feelslike, wind_speed, vehicle=None):
    a = numpyro.sample("a", dist.Normal(0, 1.5).expand([3]))
    #aweekday = numpyro.sample("aweekday", dist.Normal(0, 0.5).expand([7]))
    bfeels  = numpyro.sample("bfeels", dist.Normal(0, 0.5))
    bwspd  = numpyro.sample("bwspd", dist.Normal(0, 0.5))
    bweekday_sin = numpyro.sample("bweekdaysin", dist.Normal(0, 0.5))
    bweekday_cos = numpyro.sample("bweekdaycos", dist.Normal(0, 0.5))
    logit_p =   a[member_type] + bwspd*wind_speed + bfeels*feelslike + bweekday_sin*weekday_hour_sin + bweekday_cos*weekday_hour_cos
    numpyro.sample("vehicle", dist.Bernoulli(logits=logit_p), obs=vehicle)

m3 = MCMC(NUTS(model3), num_warmup =1000, num_samples=2000, num_chains = 4)
m3.run(random.PRNGKey(0), **dat_list)
m3.print_summary(0.95)

In [ ]:
az.plot_trace(az.from_numpyro(m3))
plt.show()

In [ ]:
az.plot_rank(az.from_numpyro(m3))
plt.show()

In [ ]:
m11_2 = AutoLaplaceApproximation(model3)
svi = SVI(
    model3,
    m11_2,
    optim.Adam(1),
    Trace_ELBO(),
    **dat_list
)
svi_result = svi.run(random.PRNGKey(0), 1000)
p11_2 = svi_result.params

In [ ]:
numpyro.util.set_platform('gpu')

In [ ]:
from jax.lib import xla_bridge
print(xla_bridge.get_backend().platform)

## Logitisc regression: member type, temperature, wind speed, kilometers ridden, travel time, rain

In [ ]:
dat_list = dict(
    vehicle = (sampled_df["vehicle_type"] != 'bike').astype(float).values,
    member_type = OrdinalEncoder().fit_transform(sampled_df.member_type.values.reshape(-1,1)).reshape(-1).astype(int),
    feelslike = sampled_df.feelslike.values,
    wind_speed = sampled_df.wind_speed.values,
    km = sampled_df.kilometers.values,
    cloudcover = sampled_df.cloudcover.values,
    uv_index = sampled_df.uv_index.values,
    humidity = sampled_df.humidity.values,
    precip = sampled_df.precip.values,
    weekday_hour_sin = sampled_df.weekday_hour_sin.values,
    weekday_hour_cos = sampled_df.weekday_hour_cos.values,
    mahalanobis_dist = sampled_df.mahalanobis_dist.values

)


def model4(member_type, weekday_hour_sin, weekday_hour_cos,feelslike, wind_speed,km, precip, humidity, cloudcover, uv_index,mahalanobis_dist, vehicle=None):
    a = numpyro.sample("a", dist.Normal(0, 1.5).expand([3]))
    #aweekday = numpyro.sample("aweekday", dist.Normal(0, 0.5).expand([2]))
    bfeels  = numpyro.sample("bfeels", dist.Normal(0, 0.5))
    bwspd  = numpyro.sample("bwspd", dist.Normal(0, 0.5))
    bkm  = numpyro.sample("bkm", dist.Normal(0, 0.5))
    brain  = numpyro.sample("brain", dist.Normal(0, 0.5))
    bhum  = numpyro.sample("bhum", dist.Normal(0, 0.5))
    buv  = numpyro.sample("buv", dist.Normal(0, 0.5))
    bcloud  = numpyro.sample("bcloud", dist.Normal(0, 0.5))
    bdist = numpyro.sample("bdist", dist.Normal(0, 0.5))
    bweekday_sin = numpyro.sample("bweekdaysin", dist.Normal(0, 0.5))
    bweekday_cos = numpyro.sample("bweekdaycos", dist.Normal(0, 0.5))
    logit_p =  a[member_type] + bwspd*wind_speed + bfeels*feelslike + bkm*km + + brain*precip + bcloud*cloudcover + buv*uv_index +bhum*humidity + bdist* mahalanobis_dist + bweekday_sin*weekday_hour_sin + bweekday_cos*weekday_hour_cos
    numpyro.sample("vehicle", dist.Bernoulli(logits=logit_p), obs=vehicle)

m4 = MCMC(NUTS(model4), num_warmup =1000, num_samples=2000, num_chains = 4)
m4.run(random.PRNGKey(0), **dat_list)
m4.print_summary(0.95)

In [ ]:
az.plot_trace(az.from_numpyro(m4))
plt.show()

In [ ]:
az.plot_pair(az.from_numpyro(m4))
plt.show()

In [ ]:
az.plot_rank(az.from_numpyro(m4))
plt.show()

In [ ]:
m11_2 = AutoLaplaceApproximation(model4)
svi = SVI(
    model4,
    m11_2,
    optim.Adam(1),
    Trace_ELBO(),
    **dat_list
)
svi_result = svi.run(random.PRNGKey(0), 1000)
p11_2 = svi_result.params
p11_2

In [ ]:
 np.array(list(set(sampled_df['new_cluster'].values.astype(int))))

## fixed effect: intercept for each cluster





In [ ]:
dat_list = dict(
    vehicle = (sampled_df["vehicle_type"] != 'bike').astype(float).values,
    cluster = sampled_df['new_cluster'].values.astype(int)
)

def model5(cluster, vehicle=None):
    a = numpyro.sample("a", dist.Normal(0, 1.5), sample_shape= np.array(list(set(cluster))).shape)
    logit_p = a[cluster]
    numpyro.sample("vehicle", dist.Bernoulli(logits=logit_p), obs=vehicle)

m5 = MCMC(NUTS(model5), num_warmup =1000, num_samples=2000, num_chains = 4)
m5.run(random.PRNGKey(0), **dat_list)
m5.print_summary(0.95)

In [ ]:
az.plot_trace(az.from_numpyro(m5))
plt.show()

## Random effect: Random intercept


In [ ]:
dat_list = dict(
    vehicle = (sampled_df["vehicle_type"] != 'bike').astype(float).values,
    cluster = sampled_df['new_cluster'].values.astype(int),
    
)

def model6(cluster, vehicle=None):
    a_bar = numpyro.sample("a_bar", dist.Normal(0, 1.5))
    sigma = numpyro.sample("sigma", dist.Exponential(1))
    a = numpyro.sample("a", dist.Normal(a_bar, sigma), sample_shape= np.array(list(set(cluster))).shape)
    logit_p = a[cluster]
    numpyro.sample("vehicle", dist.Bernoulli(logits=logit_p), obs=vehicle)

m6 = MCMC(NUTS(model6), num_warmup =1000, num_samples=2000, num_chains = 4)
m6.run(random.PRNGKey(0), **dat_list)
m6.print_summary(0.95)

In [ ]:
az.plot_trace(az.from_numpyro(m6))
plt.show()

In [ ]:
az.compare(
    {"m5": az.from_numpyro(m5), "m6": az.from_numpyro(m6)},
    ic="waic",
    scale="deviance",
)

In [ ]:
post = m6.get_samples()

In [ ]:
# show first 100 populations in the posterior
plt.subplot(xlim=(-3, 4), ylim=(0, 0.6), xlabel="log-odds choice", ylabel="Density")
for i in range(100):
    x = jnp.linspace(-3, 4, 101)
    plt.plot(
        x,
        jnp.exp(dist.Normal(post["a_bar"][i], post["sigma"][i]).log_prob(x)),
        "k",
        alpha=0.2,
    )
plt.show()

# sample 8000 imaginary tanks from the posterior distribution
idxs = random.randint(random.PRNGKey(1), (8000,), minval=0, maxval=1999)
sim_choices = dist.Normal(post["a_bar"][idxs], post["sigma"][idxs]).sample(
    random.PRNGKey(2)
)

# transform to probability and visualize
az.plot_kde(expit(sim_choices), bw=0.3)
plt.show()

##betaBinomial

In [ ]:
dat_list = dict(
    vehicle = (sampled_df["vehicle_type"] != 'bike').astype(float).values,
    member_type = OrdinalEncoder().fit_transform(sampled_df.member_type.values.reshape(-1,1)).reshape(-1).astype(int)
)

def model61(member_type, vehicle=None):
    a = numpyro.sample("a", dist.Normal(0, 1.5).expand([3]))
    phi = numpyro.sample("phi", dist.Exponential(1))
    theta = numpyro.deterministic("theta", phi + 2)
    pbar = expit(a[member_type])
    numpyro.sample("vehicle", dist.BetaBinomial(pbar * theta, (1 - pbar) * theta, 1), obs=vehicle)

m61 = MCMC(NUTS(model61), num_warmup =1000, num_samples=2000, num_chains = 4)
m61.run(random.PRNGKey(0), **dat_list)
m61.print_summary(0.95)

In [ ]:
az.plot_trace(az.from_numpyro(m61))
plt.show()

In [ ]:
post = m61.get_samples()
for gid in range(0,3):
  # draw posterior mean beta distribution
  x = jnp.linspace(0, 1, 101)
  pbar = jnp.mean(expit(post["a"][:, gid]))
  theta = jnp.mean(post["theta"])
  plt.plot(x, jnp.exp(dist.Beta(pbar * theta, (1 - pbar) * theta).log_prob(x)))
  plt.gca().set(ylabel="Density", xlabel="probability vehicle", ylim=(0, 3))

  # draw 50 beta distributions sampled from posterior
  for i in range(200):
      p = expit(post["a"][i, gid])
      theta = post["theta"][i]
      plt.plot(
          x, jnp.exp(dist.Beta(p * theta, (1 - p) * theta).log_prob(x)), "k", alpha=0.1
      )
  plt.title("distribution of member type")
  plt.show()

In [ ]:
post["theta"]

## Random effect: GP correlated structure

In [ ]:
from scipy.spatial import distance_matrix
from sklearn.neighbors import DistanceMetric
cluster_location = sampled_df[['new_cluster', 'mean_lat', 'mean_lon']].groupby('new_cluster').max()
cluster_location['mean_lat'] = np.radians(cluster_location['mean_lat'])
cluster_location['mean_lon'] = np.radians(cluster_location['mean_lon'])

distance = DistanceMetric.get_metric('haversine')
coordinates = cluster_location[['mean_lat','mean_lon']].values
cluster_dmat = distance.pairwise(coordinates)*6373
#cluster_dmat = distance_matrix(cluster_location, cluster_location, p=2)

In [ ]:
dat_list = dict(
    vehicle = (sampled_df["vehicle_type"] != 'bike').astype(float).values,
    cluster = sampled_df['new_cluster'].values.astype(int),
    Dmat = cluster_dmat
    
)

def cov_GPL2(x, sq_eta, sq_rho, sq_sigma):
    N = x.shape[0]
    K = sq_eta * jnp.exp(-sq_rho * jnp.square(x))
    K = K.at[jnp.diag_indices(N)].add(sq_sigma)
    return K


def model10(Dmat, cluster, vehicle=None):

    etasq = numpyro.sample("etasq", dist.Exponential(2))
    rhosq = numpyro.sample("rhosq", dist.Exponential(0.5))

    #k_bar = numpyro.sample("k_bar", dist.Normal(0, 1.5))
    SIGMA = cov_GPL2(Dmat, etasq, rhosq, 0.01)
    k = numpyro.sample("k", dist.MultivariateNormal(0, SIGMA))

    #a = numpyro.sample("a", dist.Normal(a_bar, sigma), sample_shape= np.array(list(set(cluster))).shape)
    logit_p = k[cluster]
    numpyro.sample("vehicle", dist.Bernoulli(logits=logit_p), obs=vehicle)

m10 = MCMC(NUTS(model10), num_warmup =1000, num_samples=2000, num_chains = 4)
m10.run(random.PRNGKey(0), **dat_list)
m10.print_summary(0.95)

In [ ]:
az.plot_trace(az.from_numpyro(m10))
plt.show()

In [ ]:
dat_list = dict(
    vehicle = (sampled_df["vehicle_type"] != 'bike').astype(float).values,
    cluster = sampled_df['new_cluster'].values.astype(int),
    Dmat = cluster_dmat
    
)

def cov_GPL2(x, sq_eta, sq_rho, sq_sigma):
    N = x.shape[0]
    K = sq_eta * jnp.exp(-sq_rho * jnp.square(x))
    K = K.at[jnp.diag_indices(N)].add(sq_sigma)
    return K


def model10(Dmat, cluster, vehicle=None):

    etasq = numpyro.sample("etasq", dist.Exponential(2))
    rhosq = numpyro.sample("rhosq", dist.Exponential(0.5))

    k_bar = numpyro.sample("k_bar", dist.Normal(0, 1.5))
    SIGMA = cov_GPL2(Dmat, etasq, rhosq, 0.01)
    k = numpyro.sample("k", dist.MultivariateNormal(k_bar, SIGMA))

    #a = numpyro.sample("a", dist.Normal(a_bar, sigma), sample_shape= np.array(list(set(cluster))).shape)
    logit_p = k[cluster]
    numpyro.sample("vehicle", dist.Bernoulli(logits=logit_p), obs=vehicle)

m10 = MCMC(NUTS(model10), num_warmup =1000, num_samples=2000, num_chains = 4)
m10.run(random.PRNGKey(0), **dat_list)
m10.print_summary(0.95)

In [ ]:
az.plot_trace(az.from_numpyro(m10))
plt.show()

In [ ]:
post = m10.get_samples()
km_max = 2
# plot the posterior median covariance function
plt.subplot(
    xlabel="distance (km)", ylabel="covariance", xlim=(0, km_max), ylim=(0,1)
)

# compute posterior mean covariance
x_seq = jnp.linspace(0, km_max, km_max*100)
pmcov = vmap(lambda x: post["etasq"] * jnp.exp(-post["rhosq"] * x**2))(x_seq)
pmcov_mu = jnp.mean(pmcov, 1)
plt.plot(x_seq, pmcov_mu, lw=2, c="k")

# plot 50 functions sampled from posterior
x = x_seq
for i in range(100):
    plt.plot(
        x_seq, post["etasq"][i] * jnp.exp(-post["rhosq"][i] * x**2), "k", alpha=0.15
    )

## Random effect: GPL1 correlated structure 

In [ ]:
dat_list = dict(
    vehicle = (sampled_df["vehicle_type"] != 'bike').astype(float).values,
    cluster = sampled_df['new_cluster'].values.astype(int),
    Dmat = cluster_dmat
    
)

def cov_GPL1(x, sq_eta, sq_rho, sq_sigma):
    N = x.shape[0]
    K = sq_eta * jnp.exp(-sq_rho * x)
    K = K.at[jnp.diag_indices(N)].add(sq_sigma)
    return K


def model101(Dmat, cluster, vehicle=None):

    etasq = numpyro.sample("etasq", dist.Exponential(2))
    rhosq = numpyro.sample("rhosq", dist.Exponential(0.5))

    k_bar = numpyro.sample("k_bar", dist.Normal(0, 1.5))
    SIGMA = cov_GPL1(Dmat, etasq, rhosq, 0.01)
    k = numpyro.sample("k", dist.MultivariateNormal(k_bar, SIGMA))

    #a = numpyro.sample("a", dist.Normal(a_bar, sigma), sample_shape= np.array(list(set(cluster))).shape)
    logit_p = k[cluster]
    numpyro.sample("vehicle", dist.Bernoulli(logits=logit_p), obs=vehicle)

m101 = MCMC(NUTS(model101), num_warmup =1000, num_samples=2000, num_chains = 4)
m101.run(random.PRNGKey(0), **dat_list)
m101.print_summary(0.95)

In [ ]:
az.plot_trace(az.from_numpyro(m101))
plt.show()

In [ ]:
post = m101.get_samples()
km_max = 6
# plot the posterior median covariance function
plt.subplot(
    xlabel="distance (km)", ylabel="covariance", xlim=(0, km_max), ylim=(0,1)
)

# compute posterior mean covariance
x_seq = jnp.linspace(0, km_max, km_max*100)
pmcov = vmap(lambda x: post["etasq"] * jnp.exp(-post["rhosq"] * x))(x_seq)
pmcov_mu = jnp.mean(pmcov, 1)
plt.plot(x_seq, pmcov_mu, lw=2, c="k")

# plot 50 functions sampled from posterior
x = x_seq
for i in range(100):
    plt.plot(
        x_seq, post["etasq"][i] * jnp.exp(-post["rhosq"][i] * x), "k", alpha=0.15
    )

## Mixed Effect: Random intercept and fixed effect for temperature

In [ ]:
dat_list = dict(
    vehicle = (sampled_df["vehicle_type"] != 'bike').astype(float).values,
    cluster = sampled_df['new_cluster'].values.astype(int),
    feelslike = sampled_df.feelslike.values,
    wind_speed = sampled_df.wind_speed.values,
    km = sampled_df.kilometers.values,
    cloudcover = sampled_df.cloudcover.values,
    uv_index = sampled_df.uv_index.values,
    humidity = sampled_df.humidity.values,
    precip = sampled_df.precip.values,
    weekday_hour_sin = sampled_df.weekday_hour_sin.values,
    weekday_hour_cos = sampled_df.weekday_hour_cos.values,
    mahalanobis_dist = sampled_df.mahalanobis_dist.values

    
)

def model11(cluster, weekday_hour_sin, weekday_hour_cos,feelslike, wind_speed,km, precip, humidity, cloudcover, uv_index,mahalanobis_dist, vehicle=None):
    a_bar = numpyro.sample("a_bar", dist.Normal(0, 1.5))
    sigma = numpyro.sample("sigma", dist.Exponential(1))
    a = numpyro.sample("a", dist.Normal(a_bar, sigma), sample_shape= np.array(list(set(cluster))).shape)
    bfeels  = numpyro.sample("bfeels", dist.Normal(0, 0.5))
    bwspd  = numpyro.sample("bwspd", dist.Normal(0, 0.5))
    bkm  = numpyro.sample("bkm", dist.Normal(0, 0.5))
    brain  = numpyro.sample("brain", dist.Normal(0, 0.5))
    bhum  = numpyro.sample("bhum", dist.Normal(0, 0.5))
    buv  = numpyro.sample("buv", dist.Normal(0, 0.5))
    bcloud  = numpyro.sample("bcloud", dist.Normal(0, 0.5))
    bdist = numpyro.sample("bdist", dist.Normal(0, 0.5))
    bweekday_sin = numpyro.sample("bweekdaysin", dist.Normal(0, 0.5))
    bweekday_cos = numpyro.sample("bweekdaycos", dist.Normal(0, 0.5))

    logit_p = a[cluster]  + bwspd*wind_speed + bfeels*feelslike + bkm*km + + brain*precip + bcloud*cloudcover + buv*uv_index +bhum*humidity + bdist* mahalanobis_dist + bweekday_sin*weekday_hour_sin + bweekday_cos*weekday_hour_cos
    numpyro.sample("vehicle", dist.Bernoulli(logits=logit_p), obs=vehicle)

m11 = MCMC(NUTS(model11), num_warmup =1000, num_samples=2000, num_chains = 4)
m11.run(random.PRNGKey(0), **dat_list)
m11.print_summary(0.95)

In [ ]:
az.plot_trace(az.from_numpyro(m11))
plt.show()

#Mixed effect: Random intercept with GP covariance plus several fixed effects

In [ ]:
dat_list = dict(
    vehicle = (sampled_df["vehicle_type"] != 'bike').astype(float).values,
    cluster = sampled_df['new_cluster'].values.astype(int),
    feelslike = sampled_df.feelslike.values,
    wind_speed = sampled_df.wind_speed.values,
    km = sampled_df.kilometers.values,
    cloudcover = sampled_df.cloudcover.values,
    uv_index = sampled_df.uv_index.values,
    humidity = sampled_df.humidity.values,
    precip = sampled_df.precip.values,
    weekday_hour_sin = sampled_df.weekday_hour_sin.values,
    weekday_hour_cos = sampled_df.weekday_hour_cos.values,
    mahalanobis_dist = sampled_df.mahalanobis_dist.values,
    Dmat = cluster_dmat
    
)

def cov_GPL2(x, sq_eta, sq_rho, sq_sigma):
    N = x.shape[0]
    K = sq_eta * jnp.exp(-sq_rho * jnp.square(x))
    K = K.at[jnp.diag_indices(N)].add(sq_sigma)
    return K


def model12(Dmat, cluster, weekday_hour_sin, weekday_hour_cos,feelslike, wind_speed,km, precip, humidity, cloudcover, uv_index,mahalanobis_dist,vehicle=None):

    etasq = numpyro.sample("etasq", dist.Exponential(2))
    rhosq = numpyro.sample("rhosq", dist.Exponential(0.5))

    k_bar = numpyro.sample("k_bar", dist.Normal(0, 1.5))
    SIGMA = cov_GPL2(Dmat, etasq, rhosq, 0.01)
    k = numpyro.sample("k", dist.MultivariateNormal(k_bar, SIGMA))
    # fixed effects
    bfeels  = numpyro.sample("bfeels", dist.Normal(0, 0.5))
    bwspd  = numpyro.sample("bwspd", dist.Normal(0, 0.5))
    bkm  = numpyro.sample("bkm", dist.Normal(0, 0.5))
    brain  = numpyro.sample("brain", dist.Normal(0, 0.5))
    bhum  = numpyro.sample("bhum", dist.Normal(0, 0.5))
    buv  = numpyro.sample("buv", dist.Normal(0, 0.5))
    bcloud  = numpyro.sample("bcloud", dist.Normal(0, 0.5))
    bdist = numpyro.sample("bdist", dist.Normal(0, 0.5))
    bweekday_sin = numpyro.sample("bweekdaysin", dist.Normal(0, 0.5))
    bweekday_cos = numpyro.sample("bweekdaycos", dist.Normal(0, 0.5))
    #a = numpyro.sample("a", dist.Normal(a_bar, sigma), sample_shape= np.array(list(set(cluster))).shape)
    logit_p = k[cluster] + bwspd*wind_speed + bfeels*feelslike + bkm*km + + brain*precip + bcloud*cloudcover + buv*uv_index +bhum*humidity + bdist* mahalanobis_dist + bweekday_sin*weekday_hour_sin + bweekday_cos*weekday_hour_cos

    numpyro.sample("vehicle", dist.Bernoulli(logits=logit_p), obs=vehicle)

m12 = MCMC(NUTS(model12), num_warmup =1000, num_samples=2000, num_chains = 4)
m12.run(random.PRNGKey(0), **dat_list)
m12.print_summary(0.95)

In [ ]:
az.plot_trace(az.from_numpyro(m12))
plt.show()

In [ ]:
post = m12.get_samples()
km_max = 2
# plot the posterior median covariance function
plt.subplot(
    xlabel="distance (km)", ylabel="covariance", xlim=(0, km_max), ylim=(0,1.1)
)

# compute posterior mean covariance
x_seq = jnp.linspace(0, km_max, km_max*100)
pmcov = vmap(lambda x: post["etasq"] * jnp.exp(-post["rhosq"] * x**2))(x_seq)
pmcov_mu = jnp.mean(pmcov, 1)
plt.plot(x_seq, pmcov_mu, lw=2, c="k")

# plot 50 functions sampled from posterior
x = x_seq
for i in range(100):
    plt.plot(
        x_seq, post["etasq"][-i] * jnp.exp(-post["rhosq"][-i] * x**2), "k", alpha=0.15
    )


In [ ]:
dat_list = dict(
    vehicle = (sampled_df["vehicle_type"] != 'bike').astype(float).values,
    cluster = sampled_df['new_cluster'].values.astype(int),
    feelslike = sampled_df.feelslike.values,
    wind_speed = sampled_df.wind_speed.values,
    km = sampled_df.kilometers.values,
    cloudcover = sampled_df.cloudcover.values,
    uv_index = sampled_df.uv_index.values,
    humidity = sampled_df.humidity.values,
    precip = sampled_df.precip.values,
    weekday_hour_sin = sampled_df.weekday_hour_sin.values,
    weekday_hour_cos = sampled_df.weekday_hour_cos.values,
    mahalanobis_dist = sampled_df.mahalanobis_dist.values,
    Dmat = cluster_dmat
    
)

def cov_GPL1(x, sq_eta, sq_rho, sq_sigma):
    N = x.shape[0]
    K = sq_eta * jnp.exp(-sq_rho * x)
    K = K.at[jnp.diag_indices(N)].add(sq_sigma)
    return K


def model13(Dmat, cluster, weekday_hour_sin, weekday_hour_cos,feelslike, wind_speed,km, precip, humidity, cloudcover, uv_index,mahalanobis_dist, vehicle=None):

    etasq = numpyro.sample("etasq", dist.Exponential(2))
    rhosq = numpyro.sample("rhosq", dist.Exponential(1.5))

    k_bar = numpyro.sample("k_bar", dist.Normal(0, 0.5))
    SIGMA = cov_GPL1(Dmat, etasq, rhosq, 0.01)
    k = numpyro.sample("k", dist.MultivariateNormal(k_bar, SIGMA))
    # fixed effects
    bfeels  = numpyro.sample("bfeels", dist.Normal(0, 0.5))
    bwspd  = numpyro.sample("bwspd", dist.Normal(0, 0.5))
    bkm  = numpyro.sample("bkm", dist.Normal(0, 0.5))
    brain  = numpyro.sample("brain", dist.Normal(0, 0.5))
    bhum  = numpyro.sample("bhum", dist.Normal(0, 0.5))
    buv  = numpyro.sample("buv", dist.Normal(0, 0.5))
    bcloud  = numpyro.sample("bcloud", dist.Normal(0, 0.5))
    bdist = numpyro.sample("bdist", dist.Normal(0, 0.5))
    bweekday_sin = numpyro.sample("bweekdaysin", dist.Normal(0, 0.5))
    bweekday_cos = numpyro.sample("bweekdaycos", dist.Normal(0, 0.5))
    #a = numpyro.sample("a", dist.Normal(a_bar, sigma), sample_shape= np.array(list(set(cluster))).shape)
    logit_p = k[cluster]  + bwspd*wind_speed + bfeels*feelslike + bkm*km + + brain*precip + bcloud*cloudcover + buv*uv_index +bhum*humidity + bdist* mahalanobis_dist + bweekday_sin*weekday_hour_sin + bweekday_cos*weekday_hour_cos

    numpyro.sample("vehicle", dist.Bernoulli(logits=logit_p), obs=vehicle)

m13 = MCMC(NUTS(model13), num_warmup =1000, num_samples=2000, num_chains = 4)
m13.run(random.PRNGKey(0), **dat_list)
m13.print_summary(0.95)

In [ ]:
az.plot_trace(az.from_numpyro(m13))
plt.show()

In [ ]:
post = m13.get_samples()
km_max =12
# plot the posterior median covariance function
plt.subplot(
    xlabel="distance (km)", ylabel="covariance", xlim=(0, km_max), ylim=(0,2.5)
)

# compute posterior mean covariance
x_seq = jnp.linspace(0, km_max, km_max*100)
pmcov = vmap(lambda x: post["etasq"] * jnp.exp(-post["rhosq"] * x))(x_seq)
pmcov_mu = jnp.mean(pmcov, 1)
plt.plot(x_seq, pmcov_mu, lw=2, c="k")

# plot 50 functions sampled from posterior
x = x_seq
for i in range(100):
    plt.plot(
        x_seq, post["etasq"][-i] * jnp.exp(-post["rhosq"][-i] * x), "k", alpha=0.15
    )

In [ ]:
az.compare(
    {"m11": az.from_numpyro(m11), "m12": az.from_numpyro(m12),"m13": az.from_numpyro(m13) },
    ic="waic",
    scale="deviance",
)

In [ ]:
az.plot_trace(az.from_numpyro(m4))
plt.show()